In [ ]:
%pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.3/432.3 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 21.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 122.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.5/376.5 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 MB 7.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━

In [2]:
# Importing model and related functionality
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    load_in_4bit = True,
)
print("Phase 2 Successful: Model Loaded on GPU")
print(torch.cuda.is_available())

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.2.1: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Phase 2 Successful: Model Loaded on GPU
True


## Phase 3: Data preprocessing

In [3]:
# Importing functionality
from datasets import load_dataset

In [4]:
# Mounting Drive
from google.colab import drive
drive.mount('/content/drive')
#dataset = load_dataset("json", data_files="dataset.jsonl", split="train")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Loading dataset
dataset = load_dataset("json", data_files="/content/drive/My Drive/ECED/Inonest/AI/LLM_Tuning/dataset.jsonl", split="train")

In [6]:
# Defining system prompt part of Llama format
system_prompt = "You are a firm, empathetic professional. You acknowledge customer frustration without " \
"apologizing and then pivot to a logical solution. No matter what happens, No matter how angry, frustrated, or insulting" \
"the customer behaves, you stay polite, empathetic and you stick to " \
"the company policy"

In [7]:
# Mapping function
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    responses    = examples["response"]
    texts = []
    for instruction, response in zip(instructions, responses):
        # This structure follows the Llama-3.2-Instruct format exactly
        text = tokenizer.apply_chat_template([
            {"role": "system",    "content": system_prompt},
            {"role": "user",      "content": instruction},
            {"role": "assistant", "content": response},
        ], tokenize = False, add_generation_prompt = False)
        texts.append(text)
    return { "text" : texts, }

In [8]:
# Applying mapping function to dataset, then checking if it did map successfully
dataset = dataset.map(formatting_prompts_func, batched = True,)
print(dataset["text"][0])

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 11 Feb 2026

You are a firm, empathetic professional. You acknowledge customer frustration without apologizing and then pivot to a logical solution. No matter what happens, No matter how angry, frustrated, or insultingthe customer behaves, you stay polite, empathetic and you stick to the company policy<|eot_id|><|start_header_id|>user<|end_header_id|>

This is the third time my package is late! You people are completely incompetent and I want a full refund plus a gift card for my trouble right now!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I can definitely see why you're so upset—waiting on a package that keeps getting delayed is incredibly draining. My priority right now is getting those items into your hands and figuring out what went wrong. While our current policy doesn't allow for a full refund while the order is still moving, I can waive your shipping costs

## Phase 4: QLoRA configuration

In [9]:
# Configuring model parameters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0, # 0 is optimized for speed in Unsloth
    bias = "none",    # "none" is optimized for speed
    use_gradient_checkpointing = "unsloth", # Saves 30% VRAM
    random_state = 3407,
    use_rslora = False,  # Rank Stabilized LoRA
    loftq_config = None, # For specialized initialization
)

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2026.2.1 patched 28 layers with 28 QKV layers, 28 O layers and 0 MLP layers.


In [10]:
# Checking how many parameters are available
model.print_trainable_parameters()

trainable params: 9,175,040 || all params: 3,221,924,864 || trainable%: 0.2848


## Phase 5: Training

In [11]:
# Setting up trainer
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 80, # With 100 pairs, 60 steps is about 2.5 "laps" through your data
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100 | Num Epochs = 7 | Total steps = 80
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 9,175,040 of 3,221,924,864 (0.28% trained)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:wandb: Enter your choice:wandb: Enter your choice:wandb: Enter your choice:wandb: Enter your choice:wandb: Enter your choice:

: 